In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)

Mounted at /content/gdrive


In [ ]:
!pip install pandasql
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as sm
import numpy as np
import matplotlib.pyplot as plt
from sqlite3 import connect
from scipy.stats.mstats import winsorize, ks_2samp
import seaborn as sns
import pandasql as ps
import scipy as scp

conn = connect(':memory:', timeout=10)
filePath = "/content/gdrive/MyDrive/FinancePaperFIMA"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=2c0651c7a35bdfc5f50ce0b75f39c9a405ec0c1230bef10ed9256da4a5266c0a
  Stored in directory: /root/.cache/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


In [ ]:
file = filePath + "/CRSPMonthly1990Through2022.csv"
crsp1990to2022 = pd.read_csv(file)
print(crsp1990to2022)
print(crsp1990to2022.columns)


<ipython-input-3-7bb977c5e1bc>:2: DtypeWarning: Columns (5,9,18,19,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp1990to2022 = pd.read_csv(file)


         PERMNO      date  NAMEENDT  SHRCD  EXCHCD   SICCD    NCUSIP TICKER  \
0         10001  19900131       NaN   11.0     3.0    4920  39040610   GFGC   
1         10001  19900228       NaN   11.0     3.0    4920  39040610   GFGC   
2         10001  19900330       NaN   11.0     3.0    4920  39040610   GFGC   
3         10001  19900430       NaN   11.0     3.0    4920  39040610   GFGC   
4         10001  19900531       NaN   11.0     3.0    4920  39040610   GFGC   
...         ...       ...       ...    ...     ...     ...       ...    ...   
2975236   93436  20211130       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975237   93436  20211231       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975238   93436  20220131       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975239   93436  20220228       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975240   93436  20220331       NaN   11.0     3.0  9999.0  88160R10   TSLA   

                     COMNAM SHRCLS  ... CFACSHR    

In [ ]:
crsp1990to2022.to_sql("crsp1990to2022", conn, if_exists='replace')
query = '''
        SELECT DISTINCT PERMNO as permno, date, RET as ret, SHROUT as shrout, ALTPRC as altprc, EXCHCD as exchcd,
        SHRCD as shrcd, SICCD as siccd, DLRET as dlret, DLSTCD as dlstcd
        FROM crsp1990to2022
        '''
crsp1990to2022 = pd.read_sql(query, conn)

crsp1990to2022['permno']=pd.to_numeric(crsp1990to2022['permno'], errors='coerce')
crsp1990to2022['date']=pd.to_datetime(crsp1990to2022['date'], format='%Y%m%d')
crsp1990to2022['ret']=pd.to_numeric(crsp1990to2022['ret'], errors='coerce')
crsp1990to2022['shrout']=pd.to_numeric(crsp1990to2022['shrout'], errors='coerce')
crsp1990to2022['altprc']=pd.to_numeric(crsp1990to2022['altprc'], errors='coerce')
crsp1990to2022['exchcd']=pd.to_numeric(crsp1990to2022['exchcd'], errors='coerce')
crsp1990to2022['shrcd']=pd.to_numeric(crsp1990to2022['shrcd'], errors='coerce')
crsp1990to2022['siccd']=pd.to_numeric(crsp1990to2022['siccd'], errors='coerce')
crsp1990to2022['dlret']=pd.to_numeric(crsp1990to2022['dlret'], errors='coerce')
crsp1990to2022['dlstcd']=pd.to_numeric(crsp1990to2022['dlstcd'], errors='coerce')

crsp1990to2022['ret'] *= 100
crsp1990to2022['dlret'] *= 100


In [ ]:
crsp2018to2022 = crsp1990to2022[crsp1990to2022['date'] >= "2018-01-01 00:00:00"]

crsp2018to2022 = crsp2018to2022[(crsp2018to2022['shrcd']==10) | (crsp2018to2022['shrcd']==11)]

In [ ]:
crsp2018to2022['market_cap'] = abs(crsp2018to2022['shrout'] * crsp2018to2022['altprc'] / 1000)
crsp2018to2022['market_cap'].replace(0, np.NaN)

2358       2.585782e+03
2359       2.509016e+03
2360       2.553262e+03
2361       2.569842e+03
2362       2.648577e+03
               ...     
2947235    1.149642e+06
2947236    1.092218e+06
2947237    9.681319e+05
2947238    8.996190e+05
2947239    1.113736e+06
Name: market_cap, Length: 194624, dtype: float64

In [ ]:
crsp2018to2022['exchange'] = ""
for i in crsp2018to2022.index:
  if crsp2018to2022['exchcd'][i] in [1,31]:
    crsp2018to2022['exchange'][i] = "NYSE"
  elif crsp2018to2022['exchcd'][i] in [2,32]:
    crsp2018to2022['exchange'][i] = "AMEX"
  elif crsp2018to2022['exchcd'][i] in [3,33]:
    crsp2018to2022['exchange'][i] = "NASDAQ"
  else:
    crsp2018to2022['exchange'][i] = "Other"
crsp2018to2022 = crsp2018to2022[crsp2018to2022['exchange'] != "Other"]


<ipython-input-7-05d956f2c751>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['exchange'][i] = "NASDAQ"
<ipython-input-7-05d956f2c751>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['exchange'][i] = "AMEX"
<ipython-input-7-05d956f2c751>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['exchange'][i] = "Other"
<ipython-input-7-05d956f2c751>:4: SettingWithCopyWarning: 
A value is trying t

In [ ]:
print(crsp2018to2022)

         permno       date      ret     shrout      altprc  exchcd  shrcd  \
2358      10026 2018-01-31  -8.8191    18678.0   138.44000     3.0   11.0   
2359      10026 2018-02-28  -2.9688    18678.0   134.33000     3.0   11.0   
2360      10026 2018-03-29   1.9951    18697.0   136.56000     3.0   11.0   
2361      10026 2018-04-30   0.6224    18702.0   137.41000     3.0   11.0   
2362      10026 2018-05-31   3.0638    18702.0   141.62000     3.0   11.0   
...         ...        ...      ...        ...         ...     ...    ...   
2947235   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0   11.0   
2947236   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0   11.0   
2947237   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0   11.0   
2947238   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0   11.0   
2947239   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0   11.0   

          siccd  dlret  dlstcd    market_cap exchange  
2358     2052.0    

In [ ]:
crsp2018to2022['ret_adj'] = ""
for i in crsp2018to2022.index:
  if pd.isnull(crsp2018to2022['dlstcd'][i]):
    crsp2018to2022['ret_adj'][i] = crsp2018to2022['ret'][i]
  elif(pd.notnull(crsp2018to2022['dlstcd'][i])) and (pd.notnull(crsp2018to2022['dlret'][i])):
    crsp2018to2022['ret_adj'][i] = crsp2018to2022['dlret'][i]
  elif(551 <= crsp2018to2022['dlret'][i] <= 574) or (crsp2018to2022['dlret'][i] in [500, 520, 580, 584]):
    crsp2018to2022['ret_adj'][i] = -30
  else:
    crsp2018to2022['ret_adj'][i] = -100

<ipython-input-9-1a63a5816ab7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['ret_adj'] = ""
<ipython-input-9-1a63a5816ab7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['ret_adj'][i] = crsp2018to2022['ret'][i]
<ipython-input-9-1a63a5816ab7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018to2022['ret_adj'][i] = -100
<ipython-input-9-1

In [ ]:
crsp2018to2022 = crsp2018to2022.reset_index()
crsp2018to2022.drop(['shrcd', 'dlret', 'dlstcd', 'index'], inplace = True, axis = 1)

print(crsp2018to2022)
print(crsp2018to2022.columns)

        permno       date      ret     shrout      altprc  exchcd   siccd  \
0        10026 2018-01-31  -8.8191    18678.0   138.44000     3.0  2052.0   
1        10026 2018-02-28  -2.9688    18678.0   134.33000     3.0  2052.0   
2        10026 2018-03-29   1.9951    18697.0   136.56000     3.0  2052.0   
3        10026 2018-04-30   0.6224    18702.0   137.41000     3.0  2052.0   
4        10026 2018-05-31   3.0638    18702.0   141.62000     3.0  2052.0   
...        ...        ...      ...        ...         ...     ...     ...   
193370   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0  9999.0   
193371   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0  9999.0   
193372   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0  9999.0   
193373   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0  9999.0   
193374   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0  9999.0   

          market_cap exchange  ret_adj  
0       2.585782e+03   NASDAQ  -8.

In [ ]:
file = filePath + "/ccmfund1990to2022.csv"
ccmfund1990to2022 = pd.read_csv(file)
print(ccmfund1990to2022)
print(ccmfund1990to2022.columns)

<ipython-input-11-62969edd8ac4>:2: DtypeWarning: Columns (2,17,19,24,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  ccmfund1990to2022 = pd.read_csv(file)


         GVKEY LINKPRIM LIID LINKTYPE  LPERMNO  LPERMCO    LINKDT LINKENDDT  \
0         1004        P    1       LU    54594    20000  19720424         E   
1         1004        P    1       LU    54594    20000  19720424         E   
2         1004        P    1       LU    54594    20000  19720424         E   
3         1004        P    1       LU    54594    20000  19720424         E   
4         1004        P    1       LU    54594    20000  19720424         E   
...        ...      ...  ...      ...      ...      ...       ...       ...   
209121  345980        P    1       LC    20333    57666  20201216         E   
209122  345980        P    1       LC    20333    57666  20201216         E   
209123  347007        P    1       LC    15533    55364  20210310         E   
209124  347085        P    1       LC    21069    58208  20210401         E   
209125  349530        P    1       LC    17324    56274  20210709         E   

        datadate   fyear  ... priusa     sic spcind

In [ ]:
ccmfund1990to2022.to_sql("ccmfund1990to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT GVKEY, LPERMNO, datadate, LINKTYPE, LINKENDDT, seq, ceq, at, lt, txditc,
        txdb, itcb, pstkrv, pstkl, pstk, indfmt, datafmt, revt, cogs, xint, xsga
        FROM ccmfund1990to2022
        '''
compustat1990to2022 = pd.read_sql(query, conn)

In [ ]:
compustat1990to2022['GVKEY'] = pd.to_numeric(compustat1990to2022['GVKEY'], errors='coerce')
compustat1990to2022['LPERMNO'] = pd.to_numeric(compustat1990to2022['LPERMNO'], errors='coerce')
compustat1990to2022['datadate'] = pd.to_datetime(compustat1990to2022['datadate'], format = '%Y%m%d')
compustat1990to2022['LINKTYPE'] = compustat1990to2022['LINKTYPE'].apply(str)
compustat1990to2022['LINKENDDT'] = pd.to_datetime(compustat1990to2022['LINKENDDT'], format = "%Y%m%d", errors = 'coerce')
compustat1990to2022['seq'] = pd.to_numeric(compustat1990to2022['seq'], errors='coerce')
compustat1990to2022['ceq'] = pd.to_numeric(compustat1990to2022['ceq'], errors='coerce')
compustat1990to2022['at'] = pd.to_numeric(compustat1990to2022['at'], errors='coerce')
compustat1990to2022['lt'] = pd.to_numeric(compustat1990to2022['lt'], errors='coerce')
compustat1990to2022['txditc'] = pd.to_numeric(compustat1990to2022['txditc'], errors='coerce')
compustat1990to2022['txdb'] = pd.to_numeric(compustat1990to2022['txdb'], errors='coerce')
compustat1990to2022['itcb'] = pd.to_numeric(compustat1990to2022['itcb'], errors='coerce')
compustat1990to2022['pstkrv'] = pd.to_numeric(compustat1990to2022['pstkrv'], errors='coerce')
compustat1990to2022['pstkl'] = pd.to_numeric(compustat1990to2022['pstkl'], errors='coerce')
compustat1990to2022['pstk'] = pd.to_numeric(compustat1990to2022['pstk'], errors='coerce')
compustat1990to2022['indfmt'] = compustat1990to2022['indfmt'].apply(str)
compustat1990to2022['datafmt'] = compustat1990to2022['datafmt'].apply(str)
compustat1990to2022['revt'] = pd.to_numeric(compustat1990to2022['revt'], errors='coerce')
compustat1990to2022['cogs'] = pd.to_numeric(compustat1990to2022['cogs'], errors='coerce')
compustat1990to2022['xint'] = pd.to_numeric(compustat1990to2022['xint'], errors='coerce')
compustat1990to2022['xsga'] = pd.to_numeric(compustat1990to2022['xsga'], errors='coerce')



In [ ]:
compustat2018to2022 = compustat1990to2022[compustat1990to2022['datadate'] >= "2017-01-01 00:00:00"]

compustat2018to2022 = compustat2018to2022[compustat2018to2022['indfmt'] == "INDL"]
compustat2018to2022 = compustat2018to2022[compustat2018to2022['datafmt'] == "STD"]

compustat2018to2022 = compustat2018to2022[(compustat2018to2022['LINKTYPE']== "LU") | (compustat2018to2022['LINKTYPE'] == "LC")]

compustat2018to2022 = compustat2018to2022[(compustat2018to2022['datadate'] <= compustat2018to2022['LINKENDDT']) | pd.isnull(compustat2018to2022['LINKENDDT'])]

In [ ]:
compustat2018to2022.to_sql("compustat2018to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT GVKEY as gvkey, LPERMNO as permno, datadate, at,
        COALESCE(seq, ceq + pstk, at - lt) + COALESCE(txditc, txdb + itcb, 0) - COALESCE(pstkrv, pstkl, pstk, 0) as book_value,
        (revt - cogs - xint - xsga) / (COALESCE(seq, ceq + pstk, at - lt) + COALESCE(txditc, txdb + itcb, 0) - COALESCE(pstkrv, pstkl, pstk, 0)) as operating_profitability
        FROM compustat2018to2022
        '''
compustat2018to2022 = pd.read_sql(query, conn)
compustat2018to2022['book_value'] = pd.to_numeric(compustat2018to2022['book_value'], errors = 'coerce')
compustat2018to2022['operating_profitability']=pd.to_numeric(compustat2018to2022['operating_profitability'],errors='coerce')

In [ ]:
compustat2018to2022['datadate'] = pd.to_datetime(compustat2018to2022['datadate'])
compustat2018to2022['year'] = compustat2018to2022['datadate'].dt.year
compustat2018to2022 = compustat2018to2022.sort_values(by = ['permno', 'datadate'], ascending = True)
compustat2018to2022 = compustat2018to2022.reset_index()
toDrop = []
for i in range(len(compustat2018to2022) - 1):
  if(compustat2018to2022['permno'][i] == compustat2018to2022['permno'][i+1]) and (compustat2018to2022['year'][i] == compustat2018to2022['year'][i+1]):
    toDrop.append(i)
compustat2018to2022.drop(toDrop, inplace = True, axis = 0)
compustat2018to2022.drop('index', inplace = True, axis = 1)

In [ ]:
print(compustat2018to2022)
print(compustat2018to2022.columns)

        gvkey  permno   datadate         at  book_value  \
0       12825   10026 2017-09-30    867.228     745.027   
1       12825   10026 2018-09-30    932.013     811.413   
2       12825   10026 2019-09-30   1019.339     895.671   
3       12825   10026 2020-09-30   1056.553     873.911   
4       12825   10026 2021-09-30   1122.219     907.232   
...       ...     ...        ...        ...         ...   
27656  184996   93436 2017-12-31  28655.372    4237.312   
27657  184996   93436 2018-12-31  29739.614    4923.243   
27658  184996   93436 2019-12-31  34309.000    6618.000   
27659  184996   93436 2020-12-31  52148.000   22376.000   
27660  184996   93436 2021-12-31  62131.000   30213.000   

       operating_profitability  year  
0                          NaN  2017  
1                          NaN  2018  
2                          NaN  2019  
3                          NaN  2020  
4                          NaN  2021  
...                        ...   ...  
27656             

In [ ]:
compustat2018to2022['investment'] = ""
compustat2018to2022.reset_index(inplace = True, drop = True)
for i in range(1, len(compustat2018to2022)):
  if compustat2018to2022['permno'][i] == compustat2018to2022['permno'][i-1]:
    compustat2018to2022['investment'][i] = compustat2018to2022['at'][i] / compustat2018to2022['at'][i-1]

<ipython-input-18-857b06891b81>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compustat2018to2022['investment'][i] = compustat2018to2022['at'][i] / compustat2018to2022['at'][i-1]
<ipython-input-18-857b06891b81>:5: RuntimeWarning: invalid value encountered in double_scalars
  compustat2018to2022['investment'][i] = compustat2018to2022['at'][i] / compustat2018to2022['at'][i-1]


In [ ]:
compustat2018to2022.replace([np.inf, -np.inf], np.nan, inplace = True)
compustat2018to2022 = compustat2018to2022.dropna()

compustat2018to2022 = compustat2018to2022.reset_index()
compustat2018to2022.drop('index', inplace = True, axis = 1)

In [ ]:
compustat2018to2022['year'] = pd.to_numeric(compustat2018to2022['year'], errors = 'coerce')
compustat2018to2022['reference_date'] = compustat2018to2022['year'] + 1
compustat2018to2022['reference_date'] = compustat2018to2022['reference_date'].apply(str)
compustat2018to2022['reference_date'] += "-06-01"
compustat2018to2022['reference_date'] = pd.to_datetime(compustat2018to2022['reference_date'])
compustat2018to2022.drop('year', inplace = True, axis = 1)

print(compustat2018to2022)
print(compustat2018to2022.columns)

        gvkey  permno   datadate         at  book_value  \
0       12096   10028 2017-12-31     13.315       7.756   
1       12096   10028 2018-12-31     13.018       8.413   
2       12096   10028 2019-12-31     27.003      11.194   
3       12096   10028 2020-12-31     40.579      17.578   
4       12096   10028 2021-12-31     59.267      27.627   
...       ...     ...        ...        ...         ...   
18077  184996   93436 2017-12-31  28655.372    4237.312   
18078  184996   93436 2018-12-31  29739.614    4923.243   
18079  184996   93436 2019-12-31  34309.000    6618.000   
18080  184996   93436 2020-12-31  52148.000   22376.000   
18081  184996   93436 2021-12-31  62131.000   30213.000   

       operating_profitability investment reference_date  
0                     0.271532                2018-06-01  
1                     0.240818   0.977694     2019-06-01  
2                     0.306593   2.074282     2020-06-01  
3                     0.392309   1.502759     2021-06-0

In [ ]:
crsp2018to2022.to_sql("crsp2018to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, date, exchange, ret_adj as ret, market_cap
        FROM crsp2018to2022
        '''
stocks2018to2022 = pd.read_sql(query, conn)

stocks2018to2022.replace([np.inf, -np.inf], np.nan, inplace = True)
stocks2018to2022 = stocks2018to2022.dropna()

In [ ]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
stocks2018to2022['reference_date'] = ""
for i in stocks2018to2022.index:
  if stocks2018to2022['date'][i].month < 6:
    stocks2018to2022['reference_date'][i] = str(stocks2018to2022['date'][i].year - 1) + "-06-01"
  else:
    stocks2018to2022['reference_date'][i] = str(stocks2018to2022['date'][i].year) + "-06-01"
stocks2018to2022['reference_date'] = pd.to_datetime(stocks2018to2022['reference_date'])

<ipython-input-22-e644673460ad>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks2018to2022['reference_date'][i] = str(stocks2018to2022['date'][i].year - 1) + "-06-01"
<ipython-input-22-e644673460ad>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks2018to2022['reference_date'][i] = str(stocks2018to2022['date'][i].year) + "-06-01"


In [ ]:
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
compustat2018to2022.to_sql("compustat2018to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018to2022.*, compustat2018to2022.gvkey, compustat2018to2022.datadate,
        compustat2018to2022.book_value
        FROM stocks2018to2022
        LEFT JOIN compustat2018to2022
        ON stocks2018to2022.permno = compustat2018to2022.permno
        AND stocks2018to2022.reference_date = compustat2018to2022.reference_date
        '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)

In [ ]:
print(stocks2018to2022)

        permno                 date exchange       ret    market_cap  \
0        10026  2018-01-31 00:00:00   NASDAQ   -8.8191  2.585782e+03   
1        10026  2018-02-28 00:00:00   NASDAQ   -2.9688  2.509016e+03   
2        10026  2018-03-29 00:00:00   NASDAQ    1.9951  2.553262e+03   
3        10026  2018-04-30 00:00:00   NASDAQ    0.6224  2.569842e+03   
4        10026  2018-05-31 00:00:00   NASDAQ    3.0638  2.648577e+03   
...        ...                  ...      ...       ...           ...   
191612   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
191613   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
191614   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
191615   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
191616   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

             reference_date     gvkey             datadate  book_value  
0       2017-06-01 00:00:00       NaN                 None    

In [ ]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
stocksmarketequity = stocks2018to2022[stocks2018to2022['date'].dt.month == 12]
stocksmarketequity['reference_date'] = stocksmarketequity['date'].dt.year + 1
stocksmarketequity['reference_date'] = stocksmarketequity['reference_date'].apply(str)
stocksmarketequity['reference_date'] += "-06-01"
stocksmarketequity['reference_date'] = pd.to_datetime(stocksmarketequity['reference_date'])
stocksmarketequity.to_sql("stocksmarketequity", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, market_cap as market_equity
        FROM stocksmarketequity
        '''
stocksmarketequity = pd.read_sql(query, conn)
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
stocksmarketequity.to_sql("stocksmarketequity", conn, if_exists = 'replace')
query = '''SELECT DISTINCT stocks2018to2022.*, stocksmarketequity.market_equity
        FROM stocks2018to2022
        LEFT JOIN stocksmarketequity
        ON stocks2018to2022.permno = stocksmarketequity.permno
        AND stocks2018to2022.reference_date = stocksmarketequity.reference_date
        '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)
stocks2018to2022 = stocks2018to2022.dropna()
stocks2018to2022 = stocks2018to2022.reset_index()
stocks2018to2022.drop('index', inplace = True, axis = 1)

<ipython-input-25-fd7977da6e23>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksmarketequity['reference_date'] = stocksmarketequity['date'].dt.year + 1
<ipython-input-25-fd7977da6e23>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksmarketequity['reference_date'] = stocksmarketequity['reference_date'].apply(str)
<ipython-input-25-fd7977da6e23>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [ ]:

stocks2018to2022['bm_ratio'] = stocks2018to2022['book_value']/stocks2018to2022['market_equity']

stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
stocksweight = stocks2018to2022[stocks2018to2022['date'].dt.month == 6]
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
stocksweight.to_sql("stocksweight", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018to2022.*, stocksweight.market_cap as market_cap_weight
        FROM stocks2018to2022
        LEFT JOIN stocksweight
        ON stocks2018to2022.permno = stocksweight.permno
        AND stocks2018to2022.reference_date = stocksweight.reference_date
        '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)

In [ ]:
stocks2018to2022.replace([np.inf, -np.inf], np.nan, inplace = True)
stocks2018to2022 = stocks2018to2022.dropna()
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
stocks2018to2022 = stocks2018to2022[stocks2018to2022['date'] >= '2018-01-01 00:00:00']

<ipython-input-27-e74d5a72aace>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])


In [ ]:
print(stocks2018to2022)

       permno       date exchange       ret    market_cap  \
0       10028 2019-06-28     AMEX   80.5555  3.500120e+01   
1       10028 2019-07-31     AMEX  -37.8461  2.175459e+01   
2       10028 2019-08-30     AMEX   55.9406  3.392424e+01   
3       10028 2019-09-30     AMEX   -4.7619  3.230880e+01   
4       10028 2019-10-31     AMEX    6.6667  3.446272e+01   
...       ...        ...      ...       ...           ...   
79507   93436 2021-11-30   NASDAQ    2.7612  1.149642e+06   
79508   93436 2021-12-31   NASDAQ   -7.6855  1.092218e+06   
79509   93436 2022-01-31   NASDAQ  -11.3609  9.681319e+05   
79510   93436 2022-02-28   NASDAQ   -7.0768  8.996190e+05   
79511   93436 2022-03-31   NASDAQ      -100  1.113736e+06   

            reference_date     gvkey             datadate  book_value  \
0      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
1      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
2      2019-06-01 00:00:00   12096.0  2018-12-31

In [ ]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
sizebreakpoints = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6) & (stocks2018to2022['exchange'] == "NYSE")]
sizebreakpoints.to_sql("sizebreakpoints", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, market_cap
        FROM sizebreakpoints
        '''
sizebreakpoints = pd.read_sql(query, conn)
sizebreakpoints['reference_date'] = pd.to_datetime(sizebreakpoints['reference_date'])
g = sizebreakpoints.groupby(['reference_date'])

In [ ]:
sizesorts = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6)]

sizesorts['size_median'] = ""
for i in sizesorts.index:
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)

Streaming output truncated to the last 5000 lines.
<ipython-input-30-0f6b76b32cf8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)
<ipython-input-30-0f6b76b32cf8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)
<ipython-input-30-0f6b76b32cf8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [ ]:
sizesorts['size_portfolio'] = ""
for i in sizesorts.index:
  if sizesorts['market_cap'][i] > sizesorts['size_median'][i]:
    sizesorts['size_portfolio'][i] = "B"
  else:
    sizesorts['size_portfolio'][i] = "S"

<ipython-input-31-beb3343ddb75>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_portfolio'] = ""
<ipython-input-31-beb3343ddb75>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_portfolio'][i] = "S"
<ipython-input-31-beb3343ddb75>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_portfolio'][i] = "B"


In [ ]:
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
sizesorts.to_sql("sizesorts", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018to2022.*, sizesorts.size_portfolio
        FROM stocks2018to2022
        LEFT JOIN sizesorts
        ON stocks2018to2022.reference_date = sizesorts.reference_date
        AND stocks2018to2022.permno = sizesorts.permno
        '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)

print(stocks2018to2022)
print(stocks2018to2022.columns)

       permno                 date exchange       ret    market_cap  \
0       10028  2019-06-28 00:00:00     AMEX   80.5555  3.500120e+01   
1       10028  2019-07-31 00:00:00     AMEX  -37.8461  2.175459e+01   
2       10028  2019-08-30 00:00:00     AMEX   55.9406  3.392424e+01   
3       10028  2019-09-30 00:00:00     AMEX   -4.7619  3.230880e+01   
4       10028  2019-10-31 00:00:00     AMEX    6.6667  3.446272e+01   
...       ...                  ...      ...       ...           ...   
79490   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
79491   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
79492   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
79493   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
79494   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

            reference_date     gvkey             datadate  book_value  \
0      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
1

In [ ]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
valuebreakpoints = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6) & (stocks2018to2022['exchange'] == "NYSE")]
valuebreakpoints.to_sql("valuebreakpoints", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, bm_ratio
        FROM valuebreakpoints
        '''
valuebreakpoints = pd.read_sql(query, conn)
valuebreakpoints['reference_date'] = pd.to_datetime(valuebreakpoints['reference_date'])
g = valuebreakpoints.groupby(['reference_date'])

In [ ]:
valuesorts = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6)]

valuesorts['value_q30'] = ""
valuesorts['value_q70'] = ""
for i in valuesorts.index:
  valuesorts['value_q30'][i] = g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.3)
  valuesorts['value_q70'][i] = g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.7)

Streaming output truncated to the last 5000 lines.
<ipython-input-34-a9499d7f4770>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valuesorts['value_q30'][i] = g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.3)
<ipython-input-34-a9499d7f4770>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valuesorts['value_q70'][i] = g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.7)
<ipython-input-34-a9499d7f4770>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [ ]:
valuesorts['value_portfolio'] = ""
for i in valuesorts.index:
  if valuesorts['bm_ratio'][i] > valuesorts['value_q70'][i]:
    valuesorts['value_portfolio'][i] = "H"
  elif valuesorts['value_q30'][i] < valuesorts['bm_ratio'][i] <= valuesorts['value_q70'][i]:
    valuesorts['value_portfolio'][i] = "M"
  else:
    valuesorts['value_portfolio'][i] = "L"

<ipython-input-35-b3917dde1a14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valuesorts['value_portfolio'] = ""
<ipython-input-35-b3917dde1a14>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valuesorts['value_portfolio'][i] = "M"
<ipython-input-35-b3917dde1a14>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valuesorts['value_portfolio'][i] = "L"
<ipython-input-35-b3917dde

In [ ]:
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
valuesorts.to_sql("valuesorts", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018to2022.*, valuesorts.value_portfolio
        FROM stocks2018to2022
        LEFT JOIN valuesorts
        ON stocks2018to2022.reference_date = valuesorts.reference_date
        AND stocks2018to2022.permno = valuesorts.permno
        '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)

print(stocks2018to2022)
print(stocks2018to2022.columns)

       permno                 date exchange       ret    market_cap  \
0       10028  2019-06-28 00:00:00     AMEX   80.5555  3.500120e+01   
1       10028  2019-07-31 00:00:00     AMEX  -37.8461  2.175459e+01   
2       10028  2019-08-30 00:00:00     AMEX   55.9406  3.392424e+01   
3       10028  2019-09-30 00:00:00     AMEX   -4.7619  3.230880e+01   
4       10028  2019-10-31 00:00:00     AMEX    6.6667  3.446272e+01   
...       ...                  ...      ...       ...           ...   
79490   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
79491   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
79492   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
79493   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
79494   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

            reference_date     gvkey             datadate  book_value  \
0      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
1

In [42]:
stocks2018to2022.to_sql("stocks2018to2022",conn, if_exists='replace')
compustat2018to2022.to_sql("compustat2018to2022",conn, if_exists='replace')
query='''
        SELECT DISTINCT stocks2018to2022.*,
        compustat2018to2022.operating_profitability, compustat2018to2022.investment
        FROM stocks2018to2022
        LEFT JOIN compustat2018to2022
        ON stocks2018to2022.permno = compustat2018to2022.permno
        AND stocks2018to2022.reference_date = compustat2018to2022.reference_date
        '''
stocks2018to2022=pd.read_sql(query, conn)
stocks2018to2022.drop('index',inplace=True,axis=1)

In [43]:
#Robustness sort
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
robustbreakpoints = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6) & (stocks2018to2022['exchange'] == "NYSE")]
robustbreakpoints.to_sql("robustbreakpoints", conn, if_exists = 'replace')
query = '''
       SELECT DISTINCT permno, reference_date, operating_profitability
       FROM robustbreakpoints
       '''
robustbreakpoints = pd.read_sql(query, conn)
robustbreakpoints['reference_date'] = pd.to_datetime(robustbreakpoints['reference_date'])
g = robustbreakpoints.groupby(['reference_date'])
robustsorts = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6)]


robustsorts['value_q30'] = ""
robustsorts['value_q70'] = ""
for i in robustsorts.index:
 robustsorts['value_q30'][i] = g.get_group(g.get_group(robustsorts['reference_date'][i]).reset_index()['reference_date'][0])['operating_profitability'].quantile(0.3)
 robustsorts['value_q70'][i] = g.get_group(g.get_group(robustsorts['reference_date'][i]).reset_index()['reference_date'][0])['operating_profitability'].quantile(0.7)
robustsorts['robust_portfolio'] = ""
for i in robustsorts.index:
 if robustsorts['operating_profitability'][i] > robustsorts['value_q70'][i]:
   robustsorts['robust_portfolio'][i] = "R"
 elif robustsorts['value_q30'][i] < robustsorts['operating_profitability'][i] <= robustsorts['value_q70'][i]:
   robustsorts['robust_portfolio'][i] = "N"
 else:
   robustsorts['robust_portfolio'][i] = "W"
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
robustsorts.to_sql("robustsorts", conn, if_exists = 'replace')
query = '''
       SELECT DISTINCT stocks2018to2022.*, robustsorts.robust_portfolio
       FROM stocks2018to2022
       LEFT JOIN robustsorts
       ON stocks2018to2022.reference_date = robustsorts.reference_date
       AND stocks2018to2022.permno = robustsorts.permno
       '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)


print(stocks2018to2022)
print(stocks2018to2022.columns)

Streaming output truncated to the last 5000 lines.
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robustsorts['value_q30'][i] = g.get_group(g.get_group(robustsorts['reference_date'][i]).reset_index()['reference_date'][0])['operating_profitability'].quantile(0.3)
<ipython-input-43-1d46630df993>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robustsorts['value_q70'][i] = g.get_group(g.get_group(robustsorts['reference_date'][i]).reset_index()['reference_date'][0])['operating_profitability'].quantile(0.7)
<ipython-input-43-1d46630df993>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

       permno                 date exchange       ret    market_cap  \
0       10028  2019-06-28 00:00:00     AMEX   80.5555  3.500120e+01   
1       10028  2019-07-31 00:00:00     AMEX  -37.8461  2.175459e+01   
2       10028  2019-08-30 00:00:00     AMEX   55.9406  3.392424e+01   
3       10028  2019-09-30 00:00:00     AMEX   -4.7619  3.230880e+01   
4       10028  2019-10-31 00:00:00     AMEX    6.6667  3.446272e+01   
...       ...                  ...      ...       ...           ...   
79490   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
79491   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
79492   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
79493   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
79494   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

            reference_date     gvkey             datadate  book_value  \
0      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
1

In [44]:
print(stocks2018to2022)
print(stocks2018to2022.columns)

       permno                 date exchange       ret    market_cap  \
0       10028  2019-06-28 00:00:00     AMEX   80.5555  3.500120e+01   
1       10028  2019-07-31 00:00:00     AMEX  -37.8461  2.175459e+01   
2       10028  2019-08-30 00:00:00     AMEX   55.9406  3.392424e+01   
3       10028  2019-09-30 00:00:00     AMEX   -4.7619  3.230880e+01   
4       10028  2019-10-31 00:00:00     AMEX    6.6667  3.446272e+01   
...       ...                  ...      ...       ...           ...   
79490   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
79491   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
79492   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
79493   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
79494   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

            reference_date     gvkey             datadate  book_value  \
0      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
1

In [1]:
#Investment sort
stocks2018to2022['date']=pd.to_datetime(stocks2018to2022['date'])
investmentbreakpoints=stocks2018to2022[(stocks2018to2022['date'].dt.month==6)&(stocks2018to2022['exchange']=="NYSE")&(stocks2018to2022['investment']!="")]
investmentbreakpoints.to_sql("investmentbreakpoints",conn, if_exists='replace')
query = '''
        SELECT DISTINCT permno, reference_date, investment
        FROM investmentbreakpoints
        '''
investmentbreakpoints =pd.read_sql(query, conn)
investmentbreakpoints['investment']=pd.to_numeric(investmentbreakpoints['investment'],errors='coerce')
investmentbreakpoints['reference_date']=pd.to_datetime(investmentbreakpoints['reference_date'])
g=investmentbreakpoints.groupby(['reference_date'])

#create new dataframe to sort values
investmentSorts=stocks2018to2022[(stocks2018to2022['date'].dt.month==6)&(stocks2018to2022['investment']!="")]
investmentSorts['investment']=pd.to_numeric(investmentSorts['investment'],errors='coerce')

#calculate new dataframe to sort values
investmentSorts['investment_q30']=""
investmentSorts['investment_q70']=""
for i in investmentSorts.index:
    investmentSorts['investment_q30'][i]=g.get_group(g.get_group(investmentSorts['reference_date'][i]).reset_index()['reference_date'][0])['investment'].quantile(0.3)
    investmentSorts['investment_q70'][i]=g.get_group(g.get_group(investmentSorts['reference_date'][i]).reset_index()['reference_date'][0])['investment'].quantile(0.7)
#sort values into conservatives neutral and aggressive portfolios
investmentSorts['investment_portfolio']=""
for i in investmentSorts.index:
    if investmentSorts['investment'][i]>investmentSorts['investment_q70'][i]:
        investmentSorts['investment_portfolio'][i]="A"
    elif investmentSorts['investment_q30'][i]<investmentSorts['investment'][i]<=investmentSorts['investment_q70'][i]:
        investmentSorts['investment_portfolio'][i]="N"
    else:
        investmentSorts['investment_portfolio'][i]="C"
#left join value portfolio back into stock dataframe
stocks2018to2022.to_sql("stocks2018to2022",conn, if_exists='replace')
investmentSorts.to_sql("investmentSorts",conn, if_exists='replace')
query ='''
        SELECT DISTINCT stocks2018to2022.*, investmentSorts.investment_portfolio
        FROM stocks2018To2022
        LEFT JOIN investmentSorts
        ON stocks2018To2022.reference_date=investmentSorts.reference_date
        AND stocks2018To2022.permno=investmentSorts.permno
        '''
stocks2018to2022=pd.read_sql(query, conn)
stocks2018to2022.drop('index',inplace=True, axis=1)
print(stocks2018to2022)
print(stocks2018to2022.columns)


NameError: ignored

In [ ]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
retsizevalue = stocks2018to2022.groupby(['date', 'size_portfolio', 'value_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retsizerobust = stocks2018to2022.groupby(['date', 'size_portfolio', 'robust_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retsizeinvestment = stocks2018to2022.groupby(['date', 'size_portfolio', 'investment_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retvaluesize = stocks2018to2022.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retrobustsize = stocks2018to2022.groupby(['date', 'robust_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retinvestmentsize = stocks2018to2022.groupby(['date', 'investment_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))

In [ ]:
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT date
        FROM stocks2018to2022
        '''
factors = pd.read_sql(query, conn)
factors['date'] = pd.to_datetime(factors['date'])

In [ ]:
#SMB portfolio
factors['smb_size'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['smb_size'][i] = np.average(retsizevalue[dateofdata]["S"]) - np.average(retsizevalue[dateofdata]["B"])

factors['smb_robust'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['smb_robust'][i] = np.average(retsizerobust[dateofdata]["S"]) - np.average(retsizerobust[dateofdata]["B"])

factors['smb_investment'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['smb_investment'][i] = np.average(retsizeinvestment[dateofdata]["S"]) - np.average(retsizeinvestment[dateofdata]["B"])

factors['smb'] = ""
for i in factors.index:
  factors['smb'][i] = np.average([factors['smb_size'][i], factors['smb_robust'][i], factors['smb_investment'][i]])

#HML portfolio
factors['hml'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['hml'][i] = np.average(retvaluesize[dateofdata]["H"]) - np.average(retvaluesize[dateofdata]["L"])

#RMW portfolio

factors['rmw'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['rmw'][i] = np.average(retrobustsize[dateofdata]["R"]) - np.average(retrobustsize[dateofdata]["W"])

#CMA portfolio

factors['cma'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['cma'][i] = np.average(retinvestmentsize[dateofdata]["C"]) - np.average(retinvestmentsize[dateofdata]["A"])

factors['mkt'] = ""
weightedavgmonthlyret = stocks2018to2022.groupby('date').apply(lambda x: np.average(pd.to_numeric(x.ret), weights = pd.to_numeric(x.market_cap_weight)))
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['mkt'][i] = weightedavgmonthlyret[dateofdata]

In [ ]:
factors['date'] = pd.to_datetime(factors['date'])
factors = factors[factors['date'] < "2022-03-31"]

factors.to_sql("factors", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT date, mkt, smb, hml, rmw, cma
        FROM factors
        '''
factors = pd.read_sql(query, conn)

In [ ]:
factors['date'] = pd.to_datetime(factors['date'])
factors = factors.sort_values(by = ['date'], ascending = True)

factors = factors.reset_index()
factors.drop('index', inplace = True, axis = 1)

print(factors)
print(factors.columns)

In [ ]:
file = filePath + "/FamaFrenchFiveFactorsMonthly.csv"
factorsff = pd.read_csv(file)
print(factorsff)
print(factorsff.columns)

In [ ]:
factorsff.to_sql("factorsff", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT Date as date, Mkt-RF as mkt_ff, SMB as smb_ff, HML as hml_ff
        FROM factorsff
        '''
factorsff = pd.read_sql(query, conn)

In [ ]:
factorsff['mkt_ff'] = pd.to_numeric(factorsff['mkt_ff'], errors = 'coerce')
factorsff['smb_ff'] = pd.to_numeric(factorsff['smb_ff'], errors = 'coerce')
factorsff['hml_ff'] = pd.to_numeric(factorsff['hml_ff'], errors = 'coerce')

In [ ]:
factors['date'] = pd.to_datetime(factors['date'])
factors['month'] = factors['date'].dt.month
factorsff['month'] = ""
for i in factorsff.index:
  factorsff['month'][i] = int(str(factorsff['date'][i])[-2] + str(factorsff['date'][i])[-1])

factors['year'] = factors['date'].dt.year
factorsff['year'] = ""
for i in factorsff.index:
  factorsff['year'][i] = int(str(factorsff['date'][i])[0:4])

In [ ]:
factors.to_sql("factors", conn, if_exists = 'replace')
factorsff.to_sql("factorsff", conn, if_exists = 'replace')
query = '''
        SELECT factors.date, factors.mkt, factors.smb, factor.hml,
        factorsff.mkt_ff, factorsff.smb_ff, factorsff.hml_ff
        FROM factors
        LEFT JOIN factorsff
        ON factors.market = factorsff.month
        AND factors.year = factorsff.year
        '''
factors = pd.read_sql(query, conn)
print(factors)

In [ ]:
factors.describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.9])

In [ ]:
ks_2samp(factors['mkt'], factors['mkt_ff'])
ks_2samp(factors['smb'], factors['smb_ff'])
ks_2samp(factors['hml'], factors['hml_ff'])

In [ ]:
factors.corr(method = 'pearson')